In [ ]:
### DATABASE DELETED ####
### LYRICS SAVED LOCALLY###

In [39]:
## importing 'mysql.connector' as mysql for convenient
import mysql.connector 
from mysql.connector import errorcode
from Lyrics import credentials #where I saved my credentials for AWS DB
import pickle
import pandas as pd

In [40]:
#Naming my database
db_name = 'project_hiphop'

In [50]:
# Connecting to the database

cnx = mysql.connector.connect(
    host = credentials.hostname,
    user = credentials.username,
    passwd = credentials.password, database = db_name
)

print(cnx) # it will print a connection object if everything is fine

#Create a cursor
cursor = cnx.cursor()

In [46]:
#Use a function to create the DB
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [48]:
#Adding the scripts for my tables creation
TABLES = {}
TABLES['lyrics_fm'] = (
    "CREATE TABLE lyrics_fm ("
    "  song_no int(11) NOT NULL AUTO_INCREMENT,"
    "  Artist varchar(50) NOT NULL,"
    "  Title varchar(200),"
    "  Album varchar(200),"
    "  Year date,"
    "  Lyrics varchar(240000) NOT NULL,"
    "  PRIMARY KEY (song_no)"
    ") ENGINE=InnoDB")

TABLES['keys_songs'] = (
    "CREATE TABLE keys_songs ("
    "  ref_no int(11) NOT NULL AUTO_INCREMENT,"
    "  Artist int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT salaries_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

# TABLES['dept_emp'] = (
#     "CREATE TABLE dept_emp ("
#     "  emp_no int(11) NOT NULL,"
#     "  dept_no char(4) NOT NULL,"
#     "  from_date date NOT NULL,"
#     "  to_date date NOT NULL,"
#     "  PRIMARY KEY (emp_no,dept_no), KEY emp_no (emp_no),"
#     "  KEY dept_no (dept_no),"
#     "  CONSTRAINT dept_emp_ibfk_1 FOREIGN KEY (emp_no) "
#     "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
#     "  CONSTRAINT dept_emp_ibfk_2 FOREIGN KEY (dept_no) "
#     "     REFERENCES departments (dept_no) ON DELETE CASCADE"
#     ") ENGINE=InnoDB")

# TABLES['dept_emp'] = (
#     "CREATE TABLE dept_emp ("
#     "  emp_no int(11) NOT NULL,"
#     "  dept_no char(4) NOT NULL,"
#     "  from_date date NOT NULL,"
#     "  to_date date NOT NULL,"
#     "  PRIMARY KEY (emp_no,dept_no), KEY emp_no (emp_no),"
#     "  KEY dept_no (dept_no),"
#     "  CONSTRAINT dept_emp_ibfk_1 FOREIGN KEY (emp_no) "
#     "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
#     "  CONSTRAINT dept_emp_ibfk_2 FOREIGN KEY (dept_no) "
#     "     REFERENCES departments (dept_no) ON DELETE CASCADE"
#     ") ENGINE=InnoDB")

In [49]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table lyrics_fm: OK


In [5]:
#Load in a Dataframe all lyrics and artist names
tracks_df = pd.read_pickle('../Data-Cleaning/tracks_df.pkl')

In [6]:
#Checking the DF
print(tracks_df.shape)
tracks_df.head()

(55010, 5)


,Artist,Title,Album,Year,Lyrics
0,Frank Ocean,Trouble,None,2012-01-01,"Go, go, go\n\n[Verse 1]\nForbidden from the be..."
1,DJ Jazzy Jeff & The Fresh Prince,Time to Chill,"He’s The DJ, I’m The Rapper",1988-03-29,[Fresh Prince]\nFor years I have been waitin a...
2,Eminem,Walk On Water/Stan/Love The Way You Lie (SNL M...,None,2017-11-18,Walk On Water\n\n[Chorus: Skylar Grey]\nI walk...
3,Machine Gun Kelly,Candy,Hotel Diablo,2019-07-05,"[Intro: Trippie Redd]\nHaha, Big 14, you know ..."
4,Big Pun,You Ain’t a Killer,Capital Punishment,1998-04-28,[Verse 1]\nThe harsh realities of life have ta...


In [51]:
## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector.connect(
    host = credentials.hostname,
    user = credentials.username,
    passwd = credentials.password, database = db_name
)

print(cnx) # it will print a connection object if everything is fine

#Create a cursor
cursor = cnx.cursor()

In [53]:
#adding lyrics into a DB
# creating column list for insertion
cols = "`,`".join([str(i) for i in tracks_df.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in tracks_df.iterrows():
    add_tracks = "INSERT INTO lyrics_fm (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(add_tracks, tuple(row))

IntegrityError: 1048 (23000): Column 'Album' cannot be null

In [ ]:
 # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()